# Seq2Seq Attention Mechanisms Comparison
## Bahdanau (Additive) vs Luong (Multiplicative) Attention

This notebook implements and compares two attention mechanisms:
1. **Bahdanau Attention** (Additive Attention)
2. **Luong Attention** (Multiplicative Attention)

For English-Portuguese machine translation.

## Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import re
import os
import io
import time
from sklearn.model_selection import train_test_split

print("TensorFlow version:", tf.__version__)

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

## Data Preprocessing Functions

In [ ]:
def unicode_to_ascii(s):
    """Convert unicode to ASCII"""
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    """Preprocess a sentence"""
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.strip()
    w = '<start> ' + w + ' <end>'
    return w

def create_dataset(path, num_examples=None):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]] 
                  for l in lines[:num_examples]]
    return zip(*word_pairs)

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

## Attention Mechanism Implementations

### 1. Bahdanau Attention (Additive)
Formula: `score(s_t, h_i) = v^T * tanh(W_s * s_t + W_h * h_i)`

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    """Bahdanau Attention (Additive)"""
    
    def __init__(self, units, name="bahdanau_attention"):
        super(BahdanauAttention, self).__init__(name=name)
        self.W1 = tf.keras.layers.Dense(units, name="W1")
        self.W2 = tf.keras.layers.Dense(units, name="W2")
        self.V = tf.keras.layers.Dense(1, name="V")
    
    def call(self, query, values):
        # query: decoder hidden state (batch, hidden)
        # values: encoder outputs (batch, seq_len, hidden)
        
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)
        ))
        
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

print("✓ Bahdanau Attention defined")

### 2. Luong Attention (Multiplicative)
Formula: `score(s_t, h_i) = s_t^T * W_a * h_i` (general)

Three variants:
- **dot**: `s_t^T * h_i`
- **general**: `s_t^T * W_a * h_i`
- **concat**: `v^T * tanh(W_a * [s_t; h_i])`

In [ ]:
class LuongAttention(tf.keras.layers.Layer):
    """Luong Attention (Multiplicative)"""
    
    def __init__(self, units, score_type='general', name="luong_attention"):
        super(LuongAttention, self).__init__(name=name)
        self.score_type = score_type
        self.units = units
        
        if score_type == 'general':
            self.W = tf.keras.layers.Dense(units, use_bias=False, name="W_general")
        elif score_type == 'concat':
            self.W = tf.keras.layers.Dense(units, name="W_concat")
            self.V = tf.keras.layers.Dense(1, name="V_concat")
    
    def call(self, query, values):
        # query: decoder hidden state (batch, hidden)
        # values: encoder outputs (batch, seq_len, hidden)
        
        query_with_time_axis = tf.expand_dims(query, 1)
        
        if self.score_type == 'dot':
            score = tf.matmul(query_with_time_axis, values, transpose_b=True)
            score = tf.squeeze(score, axis=1)
            score = tf.expand_dims(score, axis=-1)
            
        elif self.score_type == 'general':
            score = tf.matmul(query_with_time_axis, self.W(values), transpose_b=True)
            score = tf.squeeze(score, axis=1)
            score = tf.expand_dims(score, axis=-1)
            
        elif self.score_type == 'concat':
            query_tiled = tf.tile(query_with_time_axis, [1, tf.shape(values)[1], 1])
            concat = tf.concat([query_tiled, values], axis=-1)
            score = self.V(tf.nn.tanh(self.W(concat)))
        
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

print("✓ Luong Attention defined")

## Encoder and Decoder Models

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            enc_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )
    
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    
    def initialize_hidden_state(self, batch_size=None):
        if batch_size is None:
            batch_size = self.batch_sz
        return tf.zeros((batch_size, self.enc_units))

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_layer):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            dec_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = attention_layer
    
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

print("✓ Encoder and Decoder defined")

## Training Functions

In [ ]:
def loss_function(real, pred, loss_object):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

@tf.function
def train_step(inp, targ, enc_hidden, encoder, decoder, targ_lang, optimizer, loss_object):
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * inp.shape[0], 1)
        
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions, loss_object)
            dec_input = tf.expand_dims(targ[:, t], 1)
    
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

print("✓ Training functions defined")

## Load and Prepare Dataset

In [ ]:
# Configuration
BATCH_SIZE = 64
EMBEDDING_DIM = 256
UNITS = 512
NUM_EXAMPLES = 30000  # Use subset for faster training
EPOCHS = 10

# Download dataset
path_to_zip = tf.keras.utils.get_file(
    'por-eng.zip',
    origin='http://storage.googleapis.com/download.tensorflow.org/data/por-eng.zip',
    extract=True
)
path_to_file = os.path.dirname(path_to_zip) + "/por-eng/por.txt"

print(f"Dataset path: {path_to_file}")
print(f"Using {NUM_EXAMPLES} examples")

In [ ]:
# Load data
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, NUM_EXAMPLES)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
print(f"Max input length: {max_length_inp}")
print(f"Max target length: {max_length_targ}")

# Train/val split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = \
    train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=42)

print(f"\nTraining samples: {len(input_tensor_train)}")
print(f"Validation samples: {len(input_tensor_val)}")

# Vocabulary sizes
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1
print(f"\nInput vocabulary: {vocab_inp_size}")
print(f"Target vocabulary: {vocab_tar_size}")

In [ ]:
# Create TensorFlow dataset
BUFFER_SIZE = len(input_tensor_train)
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE

dataset = tf.data.Dataset.from_tensor_slices(
    (input_tensor_train, target_tensor_train)
).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

print(f"Steps per epoch: {steps_per_epoch}")

## Model 1: Bahdanau Attention
### Build and Train

In [ ]:
print("="*60)
print("CREATING BAHDANAU ATTENTION MODEL")
print("="*60)

encoder_bahdanau = Encoder(vocab_inp_size, EMBEDDING_DIM, UNITS, BATCH_SIZE)
bahdanau_attention = BahdanauAttention(UNITS)
decoder_bahdanau = Decoder(vocab_tar_size, EMBEDDING_DIM, UNITS, BATCH_SIZE, bahdanau_attention)

optimizer_bahdanau = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

print("✓ Bahdanau model created")

In [ ]:
# Train Bahdanau model
history_bahdanau = {'loss': []}

print("\nTraining Bahdanau Attention Model...")
for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder_bahdanau.initialize_hidden_state(BATCH_SIZE)
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden, encoder_bahdanau, decoder_bahdanau,
                               targ_lang, optimizer_bahdanau, loss_object)
        total_loss += batch_loss
        
        if batch % 100 == 0:
            print(f'  Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy():.4f}')
    
    epoch_loss = total_loss / steps_per_epoch
    history_bahdanau['loss'].append(epoch_loss.numpy())
    
    print(f'Epoch {epoch + 1} Loss {epoch_loss:.4f} Time: {time.time() - start:.2f}s\n')

print("✓ Bahdanau training complete")

## Model 2: Luong Attention
### Build and Train

In [ ]:
print("="*60)
print("CREATING LUONG ATTENTION MODEL")
print("="*60)

encoder_luong = Encoder(vocab_inp_size, EMBEDDING_DIM, UNITS, BATCH_SIZE)
luong_attention = LuongAttention(UNITS, score_type='general')
decoder_luong = Decoder(vocab_tar_size, EMBEDDING_DIM, UNITS, BATCH_SIZE, luong_attention)

optimizer_luong = tf.keras.optimizers.Adam()

print("✓ Luong model created")

In [ ]:
# Train Luong model
history_luong = {'loss': []}

print("\nTraining Luong Attention Model...")
for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder_luong.initialize_hidden_state(BATCH_SIZE)
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden, encoder_luong, decoder_luong,
                               targ_lang, optimizer_luong, loss_object)
        total_loss += batch_loss
        
        if batch % 100 == 0:
            print(f'  Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy():.4f}')
    
    epoch_loss = total_loss / steps_per_epoch
    history_luong['loss'].append(epoch_loss.numpy())
    
    print(f'Epoch {epoch + 1} Loss {epoch_loss:.4f} Time: {time.time() - start:.2f}s\n')

print("✓ Luong training complete")

## Compare Results

In [ ]:
# Plot training loss comparison
plt.figure(figsize=(14, 5))

# Loss curves
plt.subplot(1, 2, 1)
epochs_range = range(1, len(history_bahdanau['loss']) + 1)
plt.plot(epochs_range, history_bahdanau['loss'], 'b-o', label='Bahdanau Attention', linewidth=2)
plt.plot(epochs_range, history_luong['loss'], 'r-s', label='Luong Attention', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss Comparison', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Final loss comparison
plt.subplot(1, 2, 2)
final_losses = [history_bahdanau['loss'][-1], history_luong['loss'][-1]]
bars = plt.bar(['Bahdanau', 'Luong'], final_losses, color=['steelblue', 'coral'], edgecolor='black')
plt.ylabel('Final Loss', fontsize=12)
plt.title('Final Training Loss', fontsize=14)
for bar, loss in zip(bars, final_losses):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{loss:.4f}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig('training_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Comparison plot saved")

In [ ]:
# Summary Report
print("="*60)
print("COMPARISON RESULTS")
print("="*60)
print(f"\nBahdanau Attention:")
print(f"  Initial Loss: {history_bahdanau['loss'][0]:.4f}")
print(f"  Final Loss:   {history_bahdanau['loss'][-1]:.4f}")
print(f"  Improvement:  {history_bahdanau['loss'][0] - history_bahdanau['loss'][-1]:.4f}")

print(f"\nLuong Attention:")
print(f"  Initial Loss: {history_luong['loss'][0]:.4f}")
print(f"  Final Loss:   {history_luong['loss'][-1]:.4f}")
print(f"  Improvement:  {history_luong['loss'][0] - history_luong['loss'][-1]:.4f}")

winner = "Bahdanau" if history_bahdanau['loss'][-1] < history_luong['loss'][-1] else "Luong"
diff = abs(history_bahdanau['loss'][-1] - history_luong['loss'][-1])
print(f"\n{'='*60}")
print(f"Better Performance: {winner} (by {diff:.4f} loss)")
print("="*60)

## Key Differences

### Bahdanau (Additive) Attention:
- Uses feedforward neural network to compute attention scores
- More parameters to learn (W1, W2, V)
- Can capture more complex alignment patterns
- Computationally more expensive

### Luong (Multiplicative) Attention:
- Uses dot product or simple weight matrix for scores
- Fewer parameters (just W for 'general')
- Computationally more efficient
- Often performs similarly to Bahdanau

## Conclusion
Both attention mechanisms are effective for sequence-to-sequence tasks. The choice between them often depends on:
- Computational budget
- Dataset characteristics  
- Required model capacity